<h1 style="text-align:left;padding:12px;border-radius:4px;color:#fff;margin-top:16px;background:linear-gradient(90deg,#4B0082,#4B0082,#000);box-shadow:0 0 0px #333;">Leonor Duarte</h1>

> **Date Completed:** 2024-11-24

## Process Phase (Cleaning Data)

The "Process" phase is where the collected data is prepared for analysis. This includes tasks like connecting to data sources, cleaning and organizing the data, filtering out unnecessary information, and ensuring the data's reliability.

**Deliverable** Documentation of data cleaning and manipulation
For this phase, I used **Pandas** to clean and manipulate the data. Here’s a step-by-step breakdown of what I did:

1. Imported the monthly data files individually, since each month was stored in a separate file.
2. Merged all the monthly files into a single data frame to enable analysis for the entire year (2023).
3. Inspected the data by identifying all columns and checking their data types.
4. Looked for null values:
- Dropped any rows with null values and double-checked to ensure no nulls remained.
5. Removed unnecessary columns that didn’t add value to the analysis.
6. Checked for duplicate rows:
- Deleted any duplicates and verified that no duplicates remained.
7. Adjusted data types where needed:
- Converted the started_at and ended_at columns from “object” format to a proper datetime format.
- Extracted additional details like hour, day, month, and date to create new columns for analysis.
- Calculated the ride duration based on the difference between start and end times.
8. Standardized the order of days and months to ensure logical sequencing.
9. Created a new column, ride_duration, to capture each ride’s duration:
- Expressed the duration in minutes (excluding seconds).
- Checked for any negative values (as they would indicate errors) and removed rows with invalid durations.
10. Exported the cleaned data to a new file for further analysis.


### 1. Read each csv file individually

In [63]:
import pandas as pd

# Since the data for each month are in a seperate file, I import each file individually

jan_2023 = pd.read_csv('202301-divvy-tripdata.csv')
feb_2023 = pd.read_csv('202302-divvy-tripdata.csv')
march_2023 = pd.read_csv('202303-divvy-tripdata.csv')
april_2023 = pd.read_csv('202304-divvy-tripdata.csv')
may_2023 = pd.read_csv('202305-divvy-tripdata.csv')
june_2023 = pd.read_csv('202306-divvy-tripdata.csv')
july_2023 = pd.read_csv('202307-divvy-tripdata.csv')
august_2023 = pd.read_csv('202308-divvy-tripdata.csv')
sept_2023 = pd.read_csv('202309-divvy-tripdata.csv')
oct_2023 = pd.read_csv('202310-divvy-tripdata.csv')
nov_2023 = pd.read_csv('202311-divvy-tripdata.csv')
dec_2023 = pd.read_csv('202312-divvy-tripdata.csv')

### 2. Merge each month into one single year of 2023

In [65]:
# Merge these files into one data frame which enable me to analyze them in term of the year 2023

df = pd.concat([jan_2023, feb_2023, march_2023, april_2023, 
                may_2023, june_2023, july_2023, august_2023, 
                sept_2023, oct_2023,nov_2023, dec_2023])

### 3. Identify the information of the data frame

In [67]:
# Identify the columns and their respective data type

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5719877 entries, 0 to 224072
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 610.9+ MB


We can identify that there are a total of 12 columns, 4 of which has the data type **'float64'**, while 9 has the data type **'object'**.

### 4. Check for null values

In [70]:
# Check the amount of null values for each column

df.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    875716
start_station_id      875848
end_station_name      929202
end_station_id        929343
start_lat                  0
start_lng                  0
end_lat                 6990
end_lng                 6990
member_casual              0
dtype: int64

From the result, we can identify that there are around 70k null values for some of the columns.

In [72]:
# Delete the null values, then reconfirm that the amount of null values are now 0

df.dropna(inplace = True)
df.isnull().sum() 

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

After dropping the null values, the result now shows that there are no longer any null values.

### 5. Delete columns that are unnecessary

In [75]:
df = df.drop(['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id',
             'start_lat', 'start_lng', 'end_lat', 'end_lng'], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4331707 entries, 0 to 224072
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ride_id        object
 1   rideable_type  object
 2   started_at     object
 3   ended_at       object
 4   member_casual  object
dtypes: object(5)
memory usage: 198.3+ MB


Since there are certain columns that will not be analyze, it is better to drop them completely for the purpose of clarity and neatness, sometimes less is more.

### 6. Check for duplicated values

In [78]:
df.duplicated().sum()

0

We are all good to go since there are no duplicated values, if there are, we should drop them too.

### 7. Convert column data type 

We will be converting the column type for two columns, __**started_at**__ column and __**ended_at**__ column from the data type **"object"** to the data type **"datetime64"**

There are 2 reasons for this:
- To **extract the hour, day, month and date** to form new columns based on them
- To calculate the **ride duration**

In [82]:
# To check the current data type
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 4331707 entries, 0 to 224072
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   ride_id        object
 1   rideable_type  object
 2   started_at     object
 3   ended_at       object
 4   member_casual  object
dtypes: object(5)
memory usage: 198.3+ MB


In [83]:
import pandas as pd

# Convert the columns to datetime, coercing errors to NaT
df["started_at"] = pd.to_datetime(df["started_at"], errors='coerce')
df["ended_at"] = pd.to_datetime(df["ended_at"], errors='coerce')

# To check the updated data type
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4331707 entries, 0 to 224072
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   ride_id        object        
 1   rideable_type  object        
 2   started_at     datetime64[ns]
 3   ended_at       datetime64[ns]
 4   member_casual  object        
dtypes: datetime64[ns](2), object(3)
memory usage: 198.3+ MB


We have successfully conver the data type from **"object"** to **"datetime64"**

### 8. Create new columns that specify the hour, the day, the month, the year of each ride. 

In [86]:
df['hour'] = df['started_at'].dt.hour # Toextract the hour
df['day'] = df['started_at'].dt.day_name() # Toextract the day of the week
df['date'] = df['started_at'].dt.day #To extract the date
df['month'] = df['started_at'].dt.month_name() # To extract the month
df['year'] = df['started_at'].dt.year #To extract the year


df.head()

,ride_id,rideable_type,started_at,ended_at,member_casual,hour,day,date,month,year
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,member,20,Saturday,21,January,2023
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,member,15,Tuesday,10,January,2023
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,casual,7,Monday,2,January,2023
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,member,10,Sunday,22,January,2023
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,member,13,Thursday,12,January,2023


### 9. Check the order of the day, month

In [88]:
df.groupby(['member_casual', 'day'])['ride_id'].count() # Check order for the day

member_casual  day      
casual         Friday       227869
               Monday       175401
               Saturday     310166
               Sunday       254761
               Thursday     198931
               Tuesday      181537
               Wednesday    183092
member         Friday       400507
               Monday       386697
               Saturday     350645
               Sunday       307870
               Thursday     452660
               Tuesday      448850
               Wednesday    452721
Name: ride_id, dtype: int64

The days are not arranged in the order that we want, so we have to arrange them

In [90]:
day_ordered = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

df['day'] = pd.Categorical(df['day'], categories=day_ordered, ordered=True)
df.groupby(['member_casual', 'day'])['ride_id'].count() 

/var/folders/l7/hmlvpv4n2fxbqscfltbz2d_40000gn/T/ipykernel_18423/3152813243.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['member_casual', 'day'])['ride_id'].count()


member_casual  day      
casual         Monday       175401
               Tuesday      181537
               Wednesday    183092
               Thursday     198931
               Friday       227869
               Saturday     310166
               Sunday       254761
member         Monday       386697
               Tuesday      448850
               Wednesday    452721
               Thursday     452660
               Friday       400507
               Saturday     350645
               Sunday       307870
Name: ride_id, dtype: int64

The days are now ordered accordingly as we want, which is Monday - Friday.

In [92]:
# Now we repeat for the month

df.groupby(['member_casual', 'month'])['ride_id'].count() # Check order for the month

member_casual  month    
casual         April        110538
               August       233856
               December      36686
               February      32776
               January       29621
               July         245294
               June         219794
               March         46792
               May          177039
               November      72097
               October      130300
               September    196964
member         April        213659
               August       351063
               December     130457
               February     116784
               January      118663
               July         328664
               June         314964
               March        153655
               May          286188
               November     202701
               October      273481
               September    309671
Name: ride_id, dtype: int64

The months are not arranged in the order that we want, so we have to arrange them.

In [94]:
# Arrange the month

month_ordered = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October',
               'November', 'December']

df['month'] = pd.Categorical(df['month'], categories=month_ordered, ordered=True)
df.groupby(['member_casual', 'month'])['ride_id'].count() 

/var/folders/l7/hmlvpv4n2fxbqscfltbz2d_40000gn/T/ipykernel_18423/195444796.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['member_casual', 'month'])['ride_id'].count()


member_casual  month    
casual         January       29621
               February      32776
               March         46792
               April        110538
               May          177039
               June         219794
               July         245294
               August       233856
               September    196964
               October      130300
               November      72097
               December      36686
member         January      118663
               February     116784
               March        153655
               April        213659
               May          286188
               June         314964
               July         328664
               August       351063
               September    309671
               October      273481
               November     202701
               December     130457
Name: ride_id, dtype: int64

Perfect! The months are now arranged as wanted, which is January - December.

### 10. Create new column that calculate the duration of each ride

In [97]:
df['ride_duration'] = (df['ended_at'] - df['started_at'])/ pd.Timedelta(minutes=1) 
#Express the ride duration in terms of only minutes , and not minutes and seconds

df.head()

,ride_id,rideable_type,started_at,ended_at,member_casual,hour,day,date,month,year,ride_duration
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,member,20,Saturday,21,January,2023,10.850000
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,member,15,Tuesday,10,January,2023,8.483333
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,casual,7,Monday,2,January,2023,13.233333
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,member,10,Sunday,22,January,2023,8.766667
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,member,13,Thursday,12,January,2023,15.316667


### 11. Verify if there are negative values for the duration of ride, which is not logical

In [99]:
df[df['ride_duration'] < 0].count()

ride_id          66
rideable_type    66
started_at       66
ended_at         66
member_casual    66
hour             66
day              66
date             66
month            66
year             66
ride_duration    66
dtype: int64

Apparently there are 66 rides with negative durations which are not logical, so we have to delete them.

In [101]:
df = df[df['ride_duration'] > 0] # Remove the rows
df[df['ride_duration'] < 0].count() # Verify

ride_id          0
rideable_type    0
started_at       0
ended_at         0
member_casual    0
hour             0
day              0
date             0
month            0
year             0
ride_duration    0
dtype: int64

We reconfirmed now that the rides with negative durations have been succesfully ommitted.

In [103]:
df.sort_values(by = ['ride_duration'], ascending = True).head()

,ride_id,rideable_type,started_at,ended_at,member_casual,hour,day,date,month,year,ride_duration
116135,313A782481C750EA,classic_bike,2023-03-14 18:18:38,2023-03-14 18:18:39,member,18,Tuesday,14,March,2023,0.016667
183114,7A74F37BDE10C496,classic_bike,2023-01-03 17:22:24,2023-01-03 17:22:25,member,17,Tuesday,3,January,2023,0.016667
170985,2AFCFA391BCFE240,electric_bike,2023-12-11 16:47:18,2023-12-11 16:47:19,member,16,Monday,11,December,2023,0.016667
234797,4D6000EF6C7B7E1D,classic_bike,2023-10-17 17:18:00,2023-10-17 17:18:01,member,17,Tuesday,17,October,2023,0.016667
168477,F2D56971A4699C88,classic_bike,2023-09-18 17:39:30,2023-09-18 17:39:31,member,17,Monday,18,September,2023,0.016667


We can also verify by sorting the duration ascendingly, then identify whether the smallest value is positive.

We can see that the shortest duration which is 0.167 minute is positive, therefore all the negative values have been deleted succesfully.

### 12. Export the cleaned file.

The file has been fully cleaned according to our needs, so we can export it as a new file.

In [107]:
df.to_csv("cycle_cleaned.csv")